# ELAS v2.1 — Fits oscillatoires par sonde + Runner (sans réseau)
Lit `/content/*.csv` et ajuste un modèle $A\cos(\Omega\ln(1+z)+\phi)$ sur les **résidus** : SN, BAO, H(z), RSD. Exporte figures + JSON.

## 0) Imports & utilitaires

In [ ]:

import os, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from numpy.linalg import inv

ROOT="/content/ELAS_v2p1"; FIG=f"{ROOT}/figures"; TAB=f"{ROOT}/tables"
os.makedirs(FIG, exist_ok=True); os.makedirs(TAB, exist_ok=True)

# Cosmologie mini
c_kms=299792.458
class P:
    def __init__(self,Om=0.32,H0=67.7,gamma=0.55,rd=147.09):
        self.Om, self.H0, self.gamma, self.rd = Om, H0, gamma, rd
def E(z,p):  return np.sqrt(p.Om*(1+z)**3 + (1-p.Om))
def DH(p):   return c_kms/p.H0
def DM(z,p):
    zz=np.linspace(0,float(z),1201); return DH(p)*np.trapz(1.0/E(zz,p), zz)
def DL(z,p): return (1+z)*DM(z,p)
def Omz(z,p): return p.Om*(1+z)**3 / E(z,p)**2
def D_lin(z,p):
    zz=np.linspace(0,float(z),1201)
    g=(1+zz)/E(zz,p); num=np.trapz(g**3, zz)
    Gz=g[-1]*num; num0=np.trapz(((1+zz)/E(0,p))**3, zz); G0=((1+0)/E(0,p))*num0
    return Gz/G0
def fs8_th(z,p):
    return (Omz(z,p)**p.gamma) * D_lin(z,p)

# GLS + scan oscillation
def gls(X, y, Cinv):
    XtC = X.T @ Cinv
    beta = np.linalg.solve(XtC @ X, XtC @ y)
    r = y - X @ beta
    chi2 = float(r.T @ Cinv @ r)
    return beta, chi2, r

def scan(x, r, Cinv, W):
    best={"w":None,"Z":-1e9,"a":0.,"b":0.,"chi2":None}
    Zs=[]
    for w in W:
        X = np.column_stack([np.cos(w*x), np.sin(w*x)])
        beta, chi2, rr = gls(X, r, Cinv)
        num = float((X.T @ Cinv @ r).T @ np.linalg.solve(X.T @ Cinv @ X, (X.T @ Cinv @ r)))
        Z = num/2.0
        Zs.append(Z)
        if Z>best["Z"]: best.update({"w":float(w),"Z":float(Z),"a":float(beta[0]),"b":float(beta[1]),"chi2":float(chi2)})
    Zs=np.array(Zs); A=float(np.hypot(best["a"],best["b"])); phi=float(np.arctan2(-best["b"],best["a"]))
    best["A"]=A; best["phi"]=phi
    return Zs,best

def plot_WZ(W,Z,title,of):
    plt.figure(figsize=(8,3)); plt.plot(W,Z); plt.title(title); plt.xlabel(r"$\Omega$"); plt.ylabel("Score (rel.)")
    plt.tight_layout(); plt.savefig(of,dpi=140); plt.close()

def plot_res(x,r,w,a,b,title,of):
    plt.figure(figsize=(6,3.5)); plt.scatter(x,r,s=12,label="résidus")
    xg=np.linspace(min(x),max(x),600); yg=a*np.cos(w*xg)+b*np.sin(w*xg)
    plt.plot(xg,yg,label="fit oscill."); plt.xlabel(r"$\ln(1+z)$"); plt.ylabel("résidu"); plt.title(title); plt.legend()
    plt.tight_layout(); plt.savefig(of,dpi=140); plt.close()


## 1) SN Ia — Pantheon+SH0ES

In [ ]:

p=P()
df=pd.read_csv("/content/sn_pantheonplus_mu.csv")
z, mu = df["z"].to_numpy(float), df["mu"].to_numpy(float)
se = df["mu_err"].to_numpy(float) if "mu_err" in df.columns else np.full_like(z,0.15)
se[~np.isfinite(se)] = 0.15
C=np.diag(se**2); Cinv=inv(C+1e-12*np.eye(len(C)))
mu0=np.array([5*np.log10(DL(zz,p))+25.0 for zz in z])
one=np.ones_like(mu0); Mhat,_,_ = gls(one.reshape(-1,1), mu-mu0, Cinv)
r=(mu-mu0)-Mhat[0]*one; x=np.log(1+z); W=np.linspace(0.5,15.0,800)
Zs,best=scan(x,r,Cinv,W)
plot_WZ(W,Zs,"SN: périodogramme","{}/SN_WZ.png".format(FIG))
plot_res(x,r,best["w"],best["a"],best["b"],"SN: résidus+fit","{}/SN_residual_fit.png".format(FIG))
sn={"probe":"SN","N":int(len(z)),"Omega_best":best["w"],"Z_best":best["Z"],"A":best["A"],"phi":best["phi"],"M_fit":float(Mhat[0])}
with open("{}/SN_oscfit.json".format(TAB),"w") as f: json.dump(sn,f,indent=2)
print(sn)


## 2) BAO — DESI DR2

In [ ]:

p=P()
df=pd.read_csv("/content/bao_desi_dr2_meas.csv")
z=df["z"].to_numpy(float)
y=np.vstack([df["DM_over_rd"].to_numpy(float), df["DH_over_rd"].to_numpy(float)]).T.reshape(-1)
dm=np.array([DM(zz,p)/p.rd for zz in z]); dh=np.array([DH(p)/E(zz,p)/p.rd for zz in z])
y0=np.vstack([dm,dh]).T.reshape(-1)
import os
if os.path.exists("/content/bao_desi_dr2_cov.npy"):
    C=np.load("/content/bao_desi_dr2_cov.npy"); C=0.5*(C+C.T)+1e-8*np.eye(C.shape[0])
else:
    C=np.diag((0.03*y)**2)
Cinv=inv(C+1e-12*np.eye(len(C)))
r=y-y0; x=np.log(1+np.repeat(z,2)); W=np.linspace(0.5,15.0,800)
Zs,best=scan(x,r,Cinv,W)
plot_WZ(W,Zs,"BAO: périodogramme","{}/BAO_WZ.png".format(FIG))
plot_res(x,r,best["w"],best["a"],best["b"],"BAO: résidus whiténés+fit","{}/BAO_residual_fit.png".format(FIG))
bao={"probe":"BAO","N":int(len(y)),"Omega_best":best["w"],"Z_best":best["Z"],"A":best["A"],"phi":best["phi"]}
with open("{}/BAO_oscfit.json".format(TAB),"w") as f: json.dump(bao,f,indent=2)
print(bao)


## 3) H(z) — Chronomètres cosmiques

In [ ]:

p=P()
df=pd.read_csv("/content/cc_Hz.csv")
z=df["z"].to_numpy(float); y=df["H"].to_numpy(float)
se=df["H_err"].to_numpy(float) if "H_err" in df.columns else np.full_like(z,np.median(np.abs(y))*0.05)
se[~np.isfinite(se)] = np.median(np.abs(y))*0.05
C=np.diag(se**2); Cinv=inv(C+1e-12*np.eye(len(C)))
y0=np.array([p.H0*E(zz,p) for zz in z])
r=y-y0; x=np.log(1+z); W=np.linspace(0.5,15.0,800)
Zs,best=scan(x,r,Cinv,W)
plot_WZ(W,Zs,"H(z): périodogramme","{}/Hz_WZ.png".format(FIG))
plot_res(x,r,best["w"],best["a"],best["b"],"H(z): résidus+fit","{}/Hz_residual_fit.png".format(FIG))
hz={"probe":"Hz","N":int(len(z)),"Omega_best":best["w"],"Z_best":best["Z"],"A":best["A"],"phi":best["phi"]}
with open("{}/Hz_oscfit.json".format(TAB),"w") as f: json.dump(hz,f,indent=2)
print(hz)


## 4) RSD — fσ8(z)

In [ ]:

p=P()
df=pd.read_csv("/content/rsd_fs8.csv")
z=df["z"].to_numpy(float); y=df["fs8"].to_numpy(float)
se=df["fs8_err"].to_numpy(float) if "fs8_err" in df.columns else np.full_like(z,np.median(np.abs(y))*0.1)
se[~np.isfinite(se)] = np.median(np.abs(y))*0.1
C=np.diag(se**2); Cinv=inv(C+1e-12*np.eye(len(C)))
y0=np.array([fs8_th(zz,p) for zz in z])
q=float((y0.T@Cinv@y)/(y0.T@Cinv@y0)); r=y-q*y0
x=np.log(1+z); W=np.linspace(0.5,15.0,800)
Zs,best=scan(x,r,Cinv,W)
plot_WZ(W,Zs,"RSD: périodogramme","{}/RSD_WZ.png".format(FIG))
plot_res(x,r,best["w"],best["a"],best["b"],"RSD: résidus+fit","{}/RSD_residual_fit.png".format(FIG))
rsd={"probe":"RSD","N":int(len(z)),"Omega_best":best["w"],"Z_best":best["Z"],"A":best["A"],"phi":best["phi"],"q":float(q)}
with open("{}/RSD_oscfit.json".format(TAB),"w") as f: json.dump(rsd,f,indent=2)
print(rsd)


## 5) Résumé

In [ ]:

import glob, json, os
summ={}
for name in ["SN","BAO","Hz","RSD"]:
    p=f"{TAB}/{name}_oscfit.json"
    if os.path.exists(p):
        with open(p,'r') as f: summ[name]=json.load(f)
with open(f"{TAB}/oscfit_summary.json","w") as f: json.dump(summ,f,indent=2)
print(json.dumps(summ, indent=2))
